In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -q flatbuffers 2> /dev/null
!pip install -q mediapipe 2> /dev/null

In [ ]:
%matplotlib inline

In [ ]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import seaborn as sns
import mediapipe as mp
import matplotlib.pyplot as plt

from matplotlib import animation
from pathlib import Path
import IPython
from IPython import display
from IPython.display import HTML

import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

In [ ]:
import matplotlib as mpl

mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = False

In [ ]:
class Cfg:
    RANDOM_STATE = 2023
    INPUT_ROOT = Path('/kaggle/input/asl-signs/')
    OUTPUT_ROOT = Path('kaggle/working')
    INDEX_MAP_FILE = INPUT_ROOT / 'sign_to_prediction_index_map.json'
    TRAN_FILE = INPUT_ROOT / 'train.csv'
    INDEX = 'sequence_id'
    ROW_ID = 'row_id'

In [ ]:
print(f'cv2 version: {cv2.__version__}')
print(f'MediaPipe version: {mp.__version__}')
print(f'IPython version: {IPython.__version__}')

## Import data

In [ ]:
def read_index_map(file_path=Cfg.INDEX_MAP_FILE):
    """Reads the sign to predict as json file."""
    with open(file_path, "r") as f:
        result = json.load(f)
    return result    

def read_train(file_path=Cfg.TRAN_FILE):
    """Reads the train csv as pandas data frame."""
    return pd.read_csv(file_path).set_index(Cfg.INDEX)

def read_landmark_data_by_path(file_path, input_root=Cfg.INPUT_ROOT):
    """Reads landmak data by the given file path."""
    data = pd.read_parquet(input_root / file_path)
    return data.set_index(Cfg.ROW_ID)

def read_landmark_data_by_id(sequence_id, train_data):
    """Reads the landmark data by the given sequence id."""
    file_path = train_data.loc[sequence_id]['path']
    return read_landmark_data_by_path(file_path)

In [ ]:
train_data = read_train()
train_data.head()

___
<a id='visualization'></a>
# Visualization

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

In [ ]:
def get_random_sequence_id(train_data):
    idx = np.random.randint(0, len(train_data))
    return train_data.index[idx]

In [ ]:
def create_blank_image(height, width):
    return np.zeros((height, width, 3), np.uint8)

def draw_landmarks(
    data, 
    image, 
    frame_id, 
    landmark_type, 
    connection_type, 
    landmark_color=(255, 0, 0), 
    connection_color=(0, 20, 255), 
    thickness=1, 
    circle_radius=1
):
    """Draws landmarks"""
    df = data.groupby(['frame', 'type']).get_group((frame_id, landmark_type))
    landmarks = [landmark_pb2.NormalizedLandmark(x=lm.x, y=lm.y, z=lm.z) for idx, lm in df.iterrows()]
    landmark_list = landmark_pb2.NormalizedLandmarkList(landmark = landmarks)

    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=landmark_list, 
        connections=connection_type,
        landmark_drawing_spec=mp_drawing.DrawingSpec(
            color=landmark_color, 
            thickness=thickness, 
            circle_radius=circle_radius),
        connection_drawing_spec=mp_drawing.DrawingSpec(
            color=connection_color, 
            thickness=thickness, 
            circle_radius=circle_radius))
    return image

def draw_left_hand(data, image, frame_id):
    return draw_landmarks(
        data, 
        image, 
        frame_id, 
        landmark_type='left_hand', 
        connection_type=mp_hands.HAND_CONNECTIONS,
        landmark_color=(255, 0, 0),
        connection_color=(0, 20, 255), 
        thickness=3, 
        circle_radius=3)

def draw_right_hand(data, image, frame_id):
    return draw_landmarks(
        data, 
        image, 
        frame_id, 
        landmark_type='right_hand', 
        connection_type=mp_hands.HAND_CONNECTIONS,
        landmark_color=(255, 0, 0),
        connection_color=(0, 20, 255),
        thickness=3, 
        circle_radius=3)

def draw_face(data, image, frame_id):
    return draw_landmarks(
        data, 
        image, 
        frame_id, 
        landmark_type='face', 
        connection_type=mp_face_mesh.FACEMESH_TESSELATION,
        landmark_color=(255, 255, 255),
        connection_color=(0, 255, 0))      
    
def draw_pose(data, image, frame_id):
    return draw_landmarks(
        data, 
        image, 
        frame_id, 
        landmark_type='pose', 
        connection_type=mp_pose.POSE_CONNECTIONS,
        landmark_color=(255, 255, 255),
        connection_color=(255, 0, 0),
        thickness=2, 
        circle_radius=2)

def create_frame(data, frame_id, height=1000, width=1000):
    image = create_blank_image(height, width)    

    draw_pose(data, image, frame_id) 
    draw_left_hand(data, image, frame_id)    
    draw_right_hand(data, image, frame_id)  
    draw_face(data, image, frame_id)
     
    return image

In [ ]:
height = 800
width = 600

sequence_id = 1000106739
data = read_landmark_data_by_id(sequence_id, train_data)

frame_id = data['frame'][0]

## Hand Landmarks

<center>
<div>
    <img src="https://developers.google.com/static/mediapipe/images/solutions/hand-landmarks.png" width="600">
</div>
</center>

[Source](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker#get_started)

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(4, 4))
image = draw_right_hand(data, image=create_blank_image(height, width), frame_id=frame_id)
 
ax.imshow(image)
ax.axis('off')

plt.show()

## Face Landmarks

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(4, 4))
image = draw_face(data, image=create_blank_image(height, width), frame_id=frame_id)
 
ax.imshow(image)
ax.axis('off')

plt.show()

## Pose Landmarks

<center>
<div>
    <img src="https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png" width="600">
</div>
</center>

[Source](https://google.github.io/mediapipe/solutions/pose.html#pose-landmark-model-blazepose-ghum-3d)

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(4, 4))
image = draw_pose(data, image=create_blank_image(height, width), frame_id=frame_id)
 
ax.imshow(image)
ax.axis('off')

plt.show()

___
<a id='animation'></a>
# Animation

In [ ]:
def create_frames(sequence_id, train_data, height=800, width=800):
    data = read_landmark_data_by_id(sequence_id, train_data)
    frame_ids = data['frame'].unique()
    images = [create_frame(data, frame_id=fid, height=height, width=width) for fid in frame_ids]
    return np.array(images)

In [ ]:
def create_animation(images, fig, ax):
    ax.axis('off')
    
    ims = []
    for img in images:
        im = ax.imshow(img, animated=True)
        ims.append([im])
    
    func_animation = animation.ArtistAnimation(
        fig, 
        ims, 
        interval=100, 
        blit=True,
        repeat_delay=1000)

    return func_animation

def get_sign_by_id(sequence_id, train_data):
    return train_data.loc[sequence_id]['sign']

def play_animation(sequence_id, train_data, height, width, figsize=(4, 4)):
    frames = create_frames(sequence_id, train_data, height=height, width=width)
    sign = get_sign_by_id(sequence_id, train_data)
    
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    anim = create_animation(frames, fig, ax)
    ax.set_title(f'Sign: {sign}')
    
    video = anim.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.close()

In [ ]:
sequence_id = get_random_sequence_id(train_data)
play_animation(sequence_id, train_data, height=height, width=width)

## Sign `happy`

In [ ]:
data = train_data[train_data['sign'] == 'happy']
sequence_id = get_random_sequence_id(data)

play_animation(sequence_id, train_data, height=height, width=width)

## Sign `cow`

In [ ]:
data = train_data[train_data['sign'] == 'cow']
sequence_id = get_random_sequence_id(data)

play_animation(sequence_id, train_data, height=height, width=width)

## Sign `chocolate`

In [ ]:
data = train_data[train_data['sign'] == 'chocolate']
sequence_id = get_random_sequence_id(data)

play_animation(sequence_id, train_data, height=height, width=width)

## Sign `tiger`

In [ ]:
data = train_data[train_data['sign'] == 'tiger']
sequence_id = get_random_sequence_id(data)

play_animation(sequence_id, train_data, height=height, width=width)

## Sign `kitty`

In [ ]:
data = train_data[train_data['sign'] == 'kitty']
sequence_id = get_random_sequence_id(data)

play_animation(sequence_id, train_data, height=height, width=width)

##  Sign `moon`

In [ ]:
data = train_data[train_data['sign'] == 'moon']
sequence_id = get_random_sequence_id(data)

play_animation(sequence_id, train_data, height=height, width=width)